In [1]:
!pip install --upgrade torch
!pip install --upgrade datasets
!pip install --upgrade huggingface-hub
!pip install --upgrade accelerate
!pip install --upgrade einops
!pip install bitsandbytes==0.37.2
!pip install wandb
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 79.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 28.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 13.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 130.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 15.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 40.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
# !git clone https://github.com/andersonbcdefg/gpt.git
!cd gpt && git pull

Already up to date.


In [3]:
import os
from pynvml import *
import numpy as np
import torch
import bitsandbytes as bnb
from gpt.model import GPT, GPTConfig
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, DataCollatorForLanguageModeling
import wandb
from accelerate import Accelerator
wandb.login(key="3a67547094d7850e567db8121a1dae89fe5f1020")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so...


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")
    print(f"GPU memory free: {info.free//1024**2} MB.")
print_gpu_utilization()

GPU memory occupied: 853 MB.
GPU memory free: 81066 MB.


In [5]:
config = GPTConfig.from_yaml("gpt/config/medium.yaml")
# config.stable_embedding = False
config.checkpointing = False
config.rotary_emb = False
model = GPT(config)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")
print(config.vocab_size)

Number of parameters: ~1261M


50304


In [6]:
# from transformers import AutoConfig, GPTNeoXForCausalLM
# config = AutoConfig.from_pretrained("EleutherAI/pythia-1.4b-deduped")
# config.use_cache = False
# model = GPTNeoXForCausalLM(config)
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")

In [7]:
# compile model
# if not config.__dict__.get("checkpointing", False):
model = torch.compile(model)

In [8]:
# Tokenize the datasets
max_len = 512
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        padding=False,
        truncation=False,
        return_special_tokens_mask=False,
    ).input_ids
    result = []
    for seq in tokens:
        result.extend(seq)
        result.append(tokenizer.eos_token_id)
    result = result[:(max_len + 1) * (len(result) // (max_len + 1))]
    seqs = np.array(result).reshape(-1, max_len + 1)
    return {
        "input_ids": seqs[:, :-1],
        "targets": seqs[:, 1:]
    }

In [9]:
tokenizer.pad_token = tokenizer.eos_token
minipile_train = load_dataset("andersonbcdefg/minipile_train_tokenized", split="train")
minipile_train.set_format("pt")
minipile_val = load_dataset("andersonbcdefg/minipile_val_tokenized", split="validation")
minipile_val.set_format("pt")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2907332 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/andersonbcdefg___parquet/andersonbcdefg--minipile_train_tokenized-d27a22dbf326568d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/1352 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/andersonbcdefg___parquet/andersonbcdefg--minipile_val_tokenized-5fc55ec08b9c358d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [10]:
micro_batch_size = 32
train_dataloader = torch.utils.data.DataLoader(minipile_train, batch_size=micro_batch_size, shuffle=True, num_workers=4, pin_memory=True) #, collate_fn=collate_fn)
val_dataloader = torch.utils.data.DataLoader(minipile_val, batch_size=micro_batch_size, shuffle=False) #, collate_fn=collate_fn)

In [11]:
# Model, optimizer, scheduler
model.to(torch.device("cuda"))
max_lr = 3.0e-4
# optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=max_lr, betas=(0.9, 0.95)) #, eps=1.0e-3) -- i really don't think this works
optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr, betas=(0.9, 0.95), fused=True)
scheduler_kwargs = {
    "max_lr": max_lr,
    "total_steps": len(train_dataloader) + 10,
    "pct_start": 0.1,
    "div_factor": 5,
    "final_div_factor": 25,
    "anneal_strategy": "linear",
    "three_phase": False
}
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, **scheduler_kwargs)

In [12]:
%%capture
wandb.init(
    project="train_gpt_1.4b",
    config={
        "model": config.__dict__,
        "max_lr": max_lr,
        "scheduler": scheduler_kwargs
    }
)

In [13]:
accelerator = Accelerator(mixed_precision="bf16")
model, optimizer, train_dataloader, scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, scheduler
)

In [14]:
from transformers.modeling_outputs import CausalLMOutputWithPast

In [15]:
!ls /storage

optimizer.bin  pytorch_model.bin  random_states_0.pkl  scheduler.bin


In [ ]:
# Training loop
micro_batch_size = micro_batch_size
max_batch_size = 2048
gradient_accumulation_steps = 256 // micro_batch_size
micro_batches = 0
optimizer_steps = 0
save_every = 10000
model.train()
## TODO: skip batches with super high loss
for index, batch in enumerate(train_dataloader):
    input_ids, labels = batch["input_ids"], batch["targets"]
    if micro_batches < gradient_accumulation_steps - 1:
        with accelerator.no_sync(model):
            micro_batch_loss = model(input_ids, labels=labels)
            if isinstance(micro_batch_loss, CausalLMOutputWithPast):
                micro_batch_loss = micro_batch_loss.loss
            accelerator.backward(micro_batch_loss)
            # accelerator.clip_grad_norm_(model.parameters(), 0.5) -- next time try 1.0
        wandb.log({
            "micro_batch_loss": micro_batch_loss.item(),
            "lr": scheduler.get_last_lr()[0],
            "gradient_accumulation_steps": gradient_accumulation_steps,
            "effective_batch_size": gradient_accumulation_steps * micro_batch_size,
            "tokens": index * 512 * micro_batch_size
        })
        micro_batches += 1
    else:
        micro_batch_loss = model(input_ids, labels=labels)
        if isinstance(micro_batch_loss, CausalLMOutputWithPast):
            micro_batch_loss = micro_batch_loss.loss
        accelerator.backward(micro_batch_loss)
        # accelerator.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        micro_batches = 0
        optimizer_steps += 1
        wandb.log({
            "micro_batch_loss": micro_batch_loss.item(),
            "lr": scheduler.get_last_lr()[0],
            "gradient_accumulation_steps": gradient_accumulation_steps,
            "effective_batch_size": gradient_accumulation_steps * micro_batch_size,
            "tokens": index * 512 * micro_batch_size,
            "optimizer_steps": optimizer_steps
        })
        # take 16 steps at each effective batch size before increasing it (to speed initial learning)
        if gradient_accumulation_steps < (max_batch_size // micro_batch_size) and optimizer_steps % 16 == 0:
            gradient_accumulation_steps += 1
    
    if (index + 1) % save_every == 0:
        print("Saving checkpoint...")
        accelerator.save_state("/storage")
    
    scheduler.step()    

Saving checkpoint...
